In [6]:
import os
import numpy as np
from glob import glob
import cv2
import matplotlib.pyplot as plt
import seaborn as sb
import json
import platform
import mediapipe as mp


from eye_tracker_dataset import EyeTrackerDataset
import sys

sys.path.append("..")
from eye_tracker_auto_labeller.scripts import mediapipe_visualizer

mp_face_mesh = mp.solutions.face_mesh

def edge_list_2_path(edge_list) :
    tesel = edge_list
    change_occured = True
    while change_occured :
        change_occured = False
        for idx in range(len(tesel)) :
            target_edge = tesel[idx]
            inner_changed = False
            for e in tesel :
                if e != target_edge and len(e) < 3 :
                    edge = e
                    if target_edge[-1] == edge[0] :
                        target_edge.append(edge[-1])
                        tesel.remove(edge)
                        change_occured = True
                        inner_changed = True
                        break
                    if target_edge[0] == edge[-1] :
                        target_edge.insert(0, edge[0])
                        tesel.remove(edge)
                        change_occured = True
                        inner_changed  = True
                        break
            if inner_changed :
                break
    change_occured = True
    while change_occured :
        change_occured = False
        for idx in range(len(tesel)) :
            source_path = tesel[idx]
            inner_changed = False
            for target_path in tesel :
                if target_path == source_path :
                    continue
                if source_path[-1] == target_path[-1] :
                    target_path.reverse()
                    source_path += target_path[1:]
                    tesel.remove(target_path)
                    change_occured = True
                    inner_changed = True
                    break
                if source_path[0] == target_path[0] :
                    source_path.reverse()
                    target_path += source_path[1:]
                    tesel.remove(source_path)
                    change_occured = True
                    inner_changed = True
                    break
            if inner_changed :
                break
    return tesel


FACE_TESSELATION_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_TESSELATION
)).tolist())
FACE_OVAL_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_FACE_OVAL
)).tolist())
FACE_LIPS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LIPS
)).tolist())
FACE_LEFT_EYEBROW_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_EYEBROW
)).tolist())
FACE_LEFT_EYE_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_EYE
)).tolist())
FACE_LEFT_IRIS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_IRIS
)).tolist())
FACE_RIGHT_EYEBROW_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_EYEBROW
)).tolist())
FACE_RIGHT_EYE_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_EYE
)).tolist())
FACE_RIGHT_IRIS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_IRIS
)).tolist())

def torch_to_numpy(torch_img) :
    return (torch_img.permute([1, 2, 0]).numpy() * 255).astype(np.uint8)

def convert2imagecoord(image, array, array_idx_list) :
    image_shape = np.array([image.shape[1], image.shape[0]])
    return (array[array_idx_list][:,  :2] * image_shape).astype(int)

SYSTEM_NAME = platform.system()
if SYSTEM_NAME == "Windows" :
    DATA_ROOT_PATH = "D:/data"
if SYSTEM_NAME == "Darwin" :
    DATA_ROOT_PATH = "../eye_tracker_auto_labeller/data/"


eye_tracker_dataset = EyeTrackerDataset(data_root_path=DATA_ROOT_PATH, return_landmark=False)
eye_tracker_dataset_with_landmark = EyeTrackerDataset(data_root_path=DATA_ROOT_PATH)
eye_tracker_dataset_full = EyeTrackerDataset(data_root_path=DATA_ROOT_PATH, return_image =True)

two_D_visualizer = mediapipe_visualizer.TwoDimensionVisualizer()

print(len(eye_tracker_dataset))


9513


In [42]:
DX = 9100
LIST_IDX_TO_LOOK = FACE_LEFT_EYE_PATH_LIST[0]


data = eye_tracker_dataset_full[IDX]

face_landmark_array = data["face_landmark_array"].numpy()
full_image = torch_to_numpy(data["image"])
image_shape = np.array([full_image.shape[1], full_image.shape[0]])
mouse_position_raw = data["mouse_position"].numpy()
mouse_position = (mouse_position_raw * image_shape).astype(int)

vis_image = full_image.copy()

cv2.polylines(
    vis_image,
    [
        convert2imagecoord(full_image, face_landmark_array, LIST_IDX_TO_LOOK),
    ],
    isClosed=False,
    color=(255,255,255),
    thickness=2
)

print('ddddd')
'''
list(map(
    lambda pos : cv2.putText(
        vis_image, f"{pos}",pos,
        cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 100, 100)
    ),
    convert2imagecoord(full_image, face_landmark_array, FACE_LEFT_EYE_PATH_LIST[0])[0:3, :]
))
'''
"""
"""
list(map(
    lambda pos, list_idx : print(
        tuple(pos),
        tuple(
            cv2.cvtColor(
                np.uint8(
                    [int(120 * (list_idx / len(LIST_IDX_TO_LOOK))), 255, 255]
                ).reshape(1, 1, 3),
                cv2.COLOR_HSV2RGB
            )[0][0]
        )
    ),
    '''cv2.circle(
        vis_image, tuple(pos), 2,
        tuple(
            cv2.cvtColor(
                np.uint8(
                    [int(120 * (list_idx / len(LIST_IDX_TO_LOOK))), 255, 255]
                ).reshape(1, 1, 3),
                cv2.COLOR_HSV2RGB
            )[0][0]
        ),
        2
    ),'''
    convert2imagecoord(full_image, face_landmark_array, LIST_IDX_TO_LOOK)[0:3, :],
    range(len(LIST_IDX_TO_LOOK))
))

cv2.circle(
    vis_image,
    tuple(mouse_position),
    10,
    (255, 50, 50),
    20
)

plt.figure(figsize=(20, 20))
plt.imshow(vis_image)
plt.show()

two_D_visualizer.visualizeFace2D(
    vis_image,
    face_landmark_array
)

plt.figure(figsize=(20, 20))
plt.imshow(vis_image)
plt.show()


SyntaxError: invalid syntax (1563482773.py, line 59)

In [19]:
cv2.cvtColor(np.uint8([120, 255, 255]).reshape(1, 1, 3), cv2.COLOR_HSV2RGB)[0][0]

array([  0,   0, 255], dtype=uint8)

In [28]:
LIST_IDX_TO_LOOK = FACE_LEFT_EYE_PATH_LIST[0]
list_idx = 1


(255, 60, 0)